In [ ]:
import xarray as xr
import rioxarray as rxr
import numpy as np
import matplotlib.pyplot as plt
import io
from PIL import Image

In [ ]:
def open_netcdf(file_path):
    # Open the NetCDF file
    # dataset = nc.Dataset(file_path, mode='r')
    dataset = xr.open_dataset(file_path)
    # dataset = dataset.time.isel(time=time_step)

    return dataset

def get_extents(dataset, variable):
    # Extract the GeoTransform attribute
    geo_transform = dataset['spatial_ref'].GeoTransform.split()
    # Convert GeoTransform values to float
    geo_transform = [float(value) for value in geo_transform]
    # Calculate the extents (four corners)
    min_x = geo_transform[0]
    pixel_width = geo_transform[1]
    max_y = geo_transform[3]
    pixel_height = geo_transform[5]
    # Get the actual dimensions of the raster layer
    n_rows, n_cols = dataset[variable][0, :, :].shape
    # Calculate the coordinates of the four corners
    top_left = (min_x, max_y)
    top_right = (min_x + n_cols * pixel_width, max_y)
    bottom_left = (min_x, max_y + n_rows * pixel_height)
    bottom_right = (min_x + n_cols * pixel_width, max_y + n_rows * pixel_height)
    return top_left, top_right, bottom_left, bottom_right

In [ ]:
dataset = open_netcdf(file_path='./data/Efficiency_files/Efficiency_high_resolution_Caesium/efficiency_snow_cover.nc')

In [ ]:
dataset['Weekly_Snow_Cover']

In [ ]:
dataset['Weekly_Snow_Cover'].isel(time=0).plot()

In [ ]:
selected_data = dataset['Weekly_Snow_Cover'].sel(time='2024-01-01')
selected_data.plot()

In [ ]:
selected_data = dataset['Weekly_Snow_Cover'].sel(time='2024-02-02')
selected_data.plot()

In [ ]:
raster_layer = dataset['Weekly_Snow_Cover'].sel(time='2024-02-02')

In [ ]:
raster_layer_min = np.nanmin(raster_layer)
raster_layer_max = np.nanmax(raster_layer)

In [ ]:
variable='Weekly_Snow_Cover'
output_path='snow_raster_layer.png'
scale='time'
time_step='2024-02-02'

In [ ]:
na_mask = np.isnan(raster_layer)

if raster_layer_max > raster_layer_min:
    normalized_layer = (raster_layer - raster_layer_min) / (raster_layer_max - raster_layer_min)
else:
    normalized_layer = np.zeros_like(raster_layer)



colormap = plt.get_cmap('Blues_r')
rgba_image = colormap(normalized_layer)

rgba_image[..., 3] = np.where(na_mask, 0, 1)

rgba_image = (rgba_image * 255).astype(np.uint8)

image = Image.fromarray(rgba_image, 'RGBA')
image.save(output_path)

top_left, top_right, bottom_left, bottom_right = get_extents(dataset, variable=variable)

buffered = io.BytesIO()
image.save(buffered, format="PNG")

In [ ]:
plt.imshow(image)

# load_dotenv

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
os.getenv('path_hdf')

# Dotenv_values

In [ ]:
from dotenv import dotenv_values

config = dotenv_values(".env")  # config = {"USER": "foo", "EMAIL": "foo@example.org"}

In [ ]:
config['path_hdf']